In [1]:
# KMEANS CLUSTERING JUPYTER NOTEBOOK
# useful imports
# basic data utils
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
pal = sns.color_palette("hls", 10)

# Dimension reduction and clustering tools
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

# Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# # Model selection
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# # Metrics for how good a clustering does
# from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, classification_report


In [2]:
# Read in data
data = pd.read_csv('data/clean_housing_data_with_crime_rates.csv')
data.columns

FileNotFoundError: [Errno 2] No such file or directory: 'data/clean_housing_data_with_crime_rates.csv'

In [ ]:
# create a scatter plot of the dataset using matplotlib
# plt.scatter(data[0, :], data[1, :])
# get mean vector and covariance matrix of the dataset 
# get principal components
# pca projection and plot

In [ ]:
pca = PCA(n_components=2)
scaler = StandardScaler() 

# subset of the data dataframe with variables that are relevant 
data_sub = data[['Bedroom', 'Bathroom', 'Area', 'PPSq', 'LotArea', 'MarketEstimate',
       'RentEstimate', 'ListedPrice', 'difference_listed_vs_estimate',
       'distance_to_maj_city_miles', 'violent_crime', 'prop_crime', 'larceny',
       'vehicle_theft']]

to_scale = data_sub.columns[ data_sub.dtypes == float ]
scaled = scaler.fit_transform(data_sub[to_scale])

# print(f'numeric features: {to_scale.shape[0]}')

preproc = pd.DataFrame(scaled, columns=scaler.get_feature_names_out())

preproc

In [ ]:
# plotting helper function
def plot_components(components,x='pc1',y='pc2'):
    '''
    2d scatter plot of dimensionality reductions with labelling

    inputs:
        components (pd.DataFrame) shape is (samples x components), index is sample names
        x (str) name of column in components to plot as x axis
        y (str) name of column in components to plot as y axis
    '''
    sns.scatterplot(data=components, x=x, y=y)
    sns.despine()
    ax = plt.gca()
    for company in components.index:
        ax.annotate(company,xy=components.loc[company])
    

In [ ]:
pcs = pca.fit_transform(preproc)
print(pcs)
pcs_df = pd.DataFrame(pcs, columns=['pc1', 'pc2'])
pcs_df.index = data.index


In [ ]:
print(pcs_df)

In [ ]:
# plot the components
plot_components(pcs_df)